In [1]:
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
# Hyper parameters
batch_size = 128
nb_epoch = 10

# Parameters for MNIST dataset
nb_classes = 10

# Parameters for MLP
prob_drop_input = 0.2               # drop probability for dropout @ input layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

In [3]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_Train = np_utils.to_categorical(y_train, nb_classes)
Y_Test = np_utils.to_categorical(y_test, nb_classes)

In [4]:
# Multilayer Perceptron model
model = Sequential()
model.add(Dense(output_dim=625, input_dim=784, init='normal', activation='sigmoid', name='dense1'))
model.add(Dropout(prob_drop_input, name='dropout1'))
model.add(Dense(output_dim=625, input_dim=625, init='normal', activation='sigmoid', name='dense2'))
model.add(Dropout(prob_drop_hidden, name='dropout2'))
model.add(Dense(output_dim=10, input_dim=625, init='normal', activation='softmax', name='dense3'))
model.compile(optimizer=RMSprop(lr=0.001, rho=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 625)               490625    
_________________________________________________________________
dropout1 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense2 (Dense)               (None, 625)               391250    
_________________________________________________________________
dropout2 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense3 (Dense)               (None, 10)                6260      
Total params: 888,135
Trainable params: 888,135
Non-trainable params: 0
_______________

C:\Users\gpu\Anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, activation="sigmoid", name="dense1", units=625, kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\gpu\Anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=625, activation="sigmoid", name="dense2", units=625, kernel_initializer="normal")`
  """
C:\Users\gpu\Anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=625, activation="softmax", name="dense3", units=10, kernel_initializer="normal")`
  import sys


In [7]:
# Train
save_model(model, './model/model_mlp')
checkpoint = ModelCheckpoint(filepath='./model/weights.epoch.{epoch:02d}-val_loss.{val_loss:.2f}.hdf5', verbose=0)
history = model.fit(X_train, Y_Train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1,
                    callbacks=[checkpoint], validation_data=(X_test, Y_Test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 2944/60000 [>.............................] - ETA: 3s - loss: 0.2676 - acc: 0.9127

C:\Users\gpu\Anaconda3\envs\keras\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


60000/60000 [==============================] - 3s 58us/step - loss: 0.2480 - acc: 0.9244 - val_loss: 0.1794 - val_acc: 0.9448
Epoch 2/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.1790 - acc: 0.9446 - val_loss: 0.1388 - val_acc: 0.9582
Epoch 3/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.1419 - acc: 0.9561 - val_loss: 0.1116 - val_acc: 0.9653
Epoch 4/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.1194 - acc: 0.9636 - val_loss: 0.0964 - val_acc: 0.9698
Epoch 5/10
60000/60000 [==============================] - 4s 63us/step - loss: 0.1019 - acc: 0.9691 - val_loss: 0.0897 - val_acc: 0.9712
Epoch 6/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.0895 - acc: 0.9726 - val_loss: 0.0850 - val_acc: 0.9738
Epoch 7/10
60000/60000 [==============================] - 4s 63us/step - loss: 0.0801 - acc: 0.9756 - val_loss: 0.0777 - val_acc: 0.9760
Epoch 8/10
60000/60000 [============================

In [8]:
# Evaluate
evaluation = model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

10000/10000 [==============================] - 1s 81us/step

Summary: Loss over the test dataset: 0.07, Accuracy: 0.98


In [11]:
# Restore trained model
loaded_model = load_model('./model/model_mlp')
loaded_model.load_weights('./model/weights.epoch.09-val_loss.0.07.hdf5')
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 625)               490625    
_________________________________________________________________
dropout1 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense2 (Dense)               (None, 625)               391250    
_________________________________________________________________
dropout2 (Dropout)           (None, 625)               0         
_________________________________________________________________
dense3 (Dense)               (None, 10)                6260      
Total params: 888,135
Trainable params: 888,135
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Evaluate with loaded model
evaluation = loaded_model.evaluate(X_test, Y_Test, verbose=1)
print('\nSummary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

10000/10000 [==============================] - 1s 75us/step

Summary: Loss over the test dataset: 0.07, Accuracy: 0.98
